# 性能分析

## 查看 TopDown 计数器

我们使用 gem5_data_proc 仓库下的脚本来处理上节运行 GEM5 得到的原始数据，获取 top-down 计数器

In [ ]:
%%bash
cd .. && source env.sh

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s ${WORK_DIR}/04-gem5/02-first-run \
    -t --topdown-raw

## 查看 Cache MPKI数据

类似地，我们也可以查看 Cache MPKI 数据。上节运行的 hello-world 不具有代表性，这里我们换为使用事先运行 SPEC06 切片得到的结果

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${WORK_DIR}/04-gem5/03-perf
cd ${WORK_DIR}/04-gem5/03-perf

export PYTHONPATH=${GEM5_DATA_PROC_HOME}
ulimit -n 65535

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s $(get_asset gem5-spec-topdown-result) \
    -o gem5-cache-example.csv \
    --cache \
    > /dev/null

mkdir -p results

python3 ${GEM5_DATA_PROC_HOME}/simpoint_cpt/compute_weighted.py \
    -r gem5-cache-example.csv \
    -j $(get_asset gem5-spec-topdown-result/cluster-0-0.json) \
    -o weighted.csv \
    | tail -n 33

## SPEC06 算分

我们还可以根据切片的权重和运行结果**估算**运行完整 SPEC06 的分数

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${WORK_DIR}/04-gem5/03-perf
cd ${WORK_DIR}/04-gem5/03-perf

export PYTHONPATH=${GEM5_DATA_PROC_HOME}
ulimit -n 65535

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s $(get_asset gem5-spec-topdown-result) \
    -o gem5-score-example.csv \
    > /dev/null

mkdir -p results

python3 ${GEM5_DATA_PROC_HOME}/simpoint_cpt/compute_weighted.py \
    -r gem5-score-example.csv \
    -j $(get_asset gem5-spec-topdown-result/cluster-0-0.json) \
    --score score.csv \
    | tail -n 41